# 內容
學習搭建 Inception Block

# 目的
- 了解 Inceotion 原理
- 了解如何導入 Inception block 至 VGG-16 的架構中

In [116]:
import numpy as np
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, Input, BatchNormalization, Activation
from keras.models import Model
from keras import backend as K
from keras import layers

# 導入 Inception V2：有著 BatchNormalization 的 Convolution

In [117]:
def Conv2d_batch_normalization(_input, filters, kernel_size, padding='same', strides=(1, 1), normalizer=True, activation='relu', name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_batch_normalization'
        act_name = name + '_activation'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    
    _input = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(_input)

    if normalizer:
        _input = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(_input)
    
    if activation:
        _input = Activation(activation, name=act_name)(_input)
    
    return _input

參考 image/inception_structure.png，搭建 inception_v1_block

In [118]:
def inception_v1_block(_input, filters, channel_axis, name):
    (filters_0, filters_1, filters_2, filters_3) = filters   # filter 的數量，((64, ), (96, 128), (16, 32), (32, ))
    branch_0 = Conv2d_batch_normalization(_input, filters_0[0], (1, 1), name=name + "_branch_0")
    
    branch_1 = Conv2d_batch_normalization(_input, filters_1[0], (1, 1), name=name + "_branch_1")
    branch_1 = Conv2d_batch_normalization(branch_1, filters_1[1], (3, 3), name=name + "_branch_1_1")

    branch_2 = Conv2d_batch_normalization(_input, filters_2[0], (1, 1), name=name + "_branch_2_0")
    branch_2 = Conv2d_batch_normalization(branch_2, filters_2[1], (5, 5), name=name + "_branch_2_1")

    branch_3 = MaxPooling2D(pool_size=(3, 3), strides=1, padding="same", name=name + "_branch_3_0")(_input)
    branch_3 = Conv2d_batch_normalization(branch_3, filters_3[0], (1, 1), name=name + "_branch_3_1")
   

    _input = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name + "_concatenated")
    return _input

## 測試

In [120]:
img_input = Input(shape=(224, 224, 1))
x = inception_v1_block(_input=img_input, filters=((64,), (96,128), (16,32), (32,)), channel_axis=3, name='block_1')
print(x)

Tensor("block_1_concatenated_8/concat:0", shape=(None, 224, 224, 256), dtype=float32)


將 inception_v1_block中的 n × n kernel 改為 1 × n + n × 1

In [121]:
def inception_v3_block(_input, filters, channel_axis, name):
    (filters_0, filters_1, filters_2, filters_3) = filters   # filter 的數量 ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_batch_normalization(_input, filters_0[0], (1, 1), name=name + "_branch_0")

    branch_1 = Conv2d_batch_normalization(_input, filters_1[0], (1, 1), name=name + "_branch_1")
    branch_1 = Conv2d_batch_normalization(branch_1, filters_1[1], (1, 3), name=name + "_branch_1_1")
    branch_1 = Conv2d_batch_normalization(branch_1, filters_1[1], (3, 1), name=name + "_branch_1_2")

    branch_2 = Conv2d_batch_normalization(_input, filters_2[0], (1, 1), name=name + "_branch_2_0")
    branch_2 = Conv2d_batch_normalization(branch_2, filters_2[1], (1, 5), name=name + "_branch_2_1")
    branch_2 = Conv2d_batch_normalization(branch_2, filters_2[1], (5, 1), name=name + "_branch_2_2")

    branch_3 = MaxPooling2D(pool_size=(3, 3), strides=1, padding="same", name=name + "_branch_3_0")(_input)
    branch_3 = Conv2d_batch_normalization(branch_3, filters_3[0], (1, 1), name=name + "_branch_3_1")

    _input = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name + "_concatenated")
    return _input

測試

In [122]:
img_input = Input(shape=(224, 224, 1))
x = inception_v3_block(_input=img_input, filters=((64,), (96,128), (16,32), (32,)), channel_axis=3, name='block_1')
print(x)

Tensor("block_1_concatenated_9/concat:0", shape=(None, 224, 224, 256), dtype=float32)


---

# 額外練習

#### 將 VGG-16 Block_3 中的 Convolution 全部改為 inception_v1_block
#### Block_5 中的 Convolution 全部改為 inception_v3_block
#### 並將所有 Convolution 改為 Conv2d_batch_normalization

# 原 VGG-16 架構

In [123]:


def vgg_16(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



#### 修改後

In [125]:
def vgg_16_inception(include_top=True,input_tensor=None, input_shape=(224,224,1), pooling='max',classes=1000):
    img_input = Input(shape=input_shape)
    
    # Block 1
    x = Conv2d_batch_normalization(img_input, 64, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = Conv2d_batch_normalization(x, 64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2d_batch_normalization(x, 128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2d_batch_normalization(x, 128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = inception_v1_block(_input=x, filters=((64,), (96,128), (16,32), (32,)), channel_axis=3, name='block_3')
    
    # Block 4
    x = Conv2d_batch_normalization(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2d_batch_normalization(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2d_batch_normalization(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = inception_v3_block(_input=x, filters=((64,), (96,128), (16,32), (32,)), channel_axis=3, name='block_5')
    
    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
    
    # Create model.
    model = Model(img_input, x, name='vgg_16_with_batch_normalization')

    
   
    return model



In [127]:
model = vgg_16_inception(include_top=False)

Tensor("block1_conv1_activation_26/Relu:0", shape=(None, 224, 224, 64), dtype=float32)
Tensor("input_51:0", shape=(None, 224, 224, 1), dtype=float32)
Tensor("global_max_pooling2d_16/Max:0", shape=(None, 256), dtype=float32)


In [130]:
model.summary()

Model: "vgg_16_with_batch_normalization"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_51[0][0]                   
__________________________________________________________________________________________________
block1_conv1_batch_normalizatio (None, 224, 224, 64) 192         block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
block1_conv1_activation (Activa (None, 224, 224, 64) 0           block1_conv1_batch_normalization[
____________________________________________________________________